# Question 1

In this question we will
- Practice reading image filenames from a .csv file for processing.
- Compare the PSNR and SSIM of different denoising methods applied to a dataset at different noise levels.




In [3]:
# Boilerplate imports
import skimage.io as skio
import skimage.color as skcolor
import skimage.util as skutil
import skimage.restoration as skrest
import skimage.filters as skfilt
import skimage.metrics as skmetrics
import os
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

# Step 1: Read the image filenames from the .csv file.  

For this you should add `import pandas as pd` to the list of boilerplate imports, above.  Then use the pd.read_csv() function and give the path to the images.csv file as an argument, and also give the optional argument `headers=None` because the given .csv file does not have column title headers.  This will return a Pandas dataframe with one column containing the filenames.  We could iterate over the dataframe, but in order to not have to get too deep into the use of Pandas dataframes, we can convert the single column dataframe to a Python list easily.  If the dataframe is referred to by a variable called `files`, we can convert the first column to a list using:  `files[0].tolist()`.  This will return a list of strings, one for each filename that we can now easily iterate over using a for-loop.


In [7]:
# Write the code for this step here
files = pd.read_csv("/Users/kanwarsingh/Desktop/Fall 2024/CMPT 487/asn1/images/images.csv",header=None)

file_list = files[0].tolist()

# Step 2: Process the images

## Instructions

- For each filename in `file_list` defined in the prevous block (for testing purposes you can use a small number of the images, say the first 3-5 filenames):
    - read in the noiseless version of that image from the images/noiseless directory.  Convert it to grayscale (since these are RGB colour images) and convert it to a dtype float64 numpy array using `skutil.img_as_float()`.
    - use the `skutil.random_noise()` function to corrupt the noiseless image with Gaussian noise with standard deviation sigma for each of the sigma values given in the array `sigmas`, defined below.  Keep in mind that the values in `sigmas` are standard deviations, and the `random_noise()` function expects *variance*, so you will need to square the sigma values when you pass them to `random_noise()`.
    - Denoise each of the 5 noisy versions of the image with Gaussian filtering (`skfilt.gaussian()`), total variation minimization (TVM) (`skrest.denoise_tv_bregman()`), and non-local means filtering (NLM) (`skrest.denoise_nl_means()`).  For Gaussian filtering, use a 7x7 filter.  For TVM and NLM just pass in the image and use the *default* parameters.
    - For each denoised image, compute and store the PSNR and SSIM resulting from denoising with each method by comparing each denoised image to the noiseless image.  Do this in such a way that is easy to compute the averages described below.
      
- Compute the average PSNR and SSIM for each denoising method and noise level.   We recommend that the results of this computation be stored in the form of six different arrays or lists, one for each combination of denoising method and metric.
- See functions `skmetrics.peak_signal_noise_ratio()` and `skmetrics.structural_similarity()` to compute PSNR and SSIM.

For testing use only the first three to five filenames in `file_list` so that you can perform testing quickly.  Once you have completed and thoroughly tested your code, run it on the entire list of filenames.  This will take several minutes.  


In [12]:
# Define the set of different noise levels (STANDARD DEVIATIONS)
sigmas = [0.01, 0.02, 0.05, 0.1, 0.2]

# Preload all noiseless images once and convert them to grayscale float images
noiseless_images = [skcolor.rgb2gray(skutil.img_as_float(skio.imread("/Users/kanwarsingh/Desktop/Fall 2024/CMPT 487/asn1/images/images/noiseless/" + filename)))
                    for filename in file_list]

# Preallocate lists for corrupted images, and apply noise in a single pass
corrupted_images = []
for noiseless_image in noiseless_images:
    for sigma in sigmas:
        corrupted_images.append(skutil.random_noise(noiseless_image, var=sigma ** 2))

# Apply denoising filters once for all corrupted images
gaus_filtered_img = [skfilt.gaussian(image, sigma=7/6) for image in corrupted_images]
tvm_filt_img = [skrest.denoise_tv_bregman(image, weight=5.0) for image in corrupted_images]
nlm_filt_img = [skrest.denoise_nl_means(image) for image in corrupted_images]

# Initialize lists for PSNR and SSIM results
avg_psnr_gaus, avg_psnr_tvm, avg_psnr_nlm = [], [], []
avg_ssim_gaus, avg_ssim_tvm, avg_ssim_nlm = [], [], []

# Group corrupted and filtered images by original noiseless image
num_images = len(noiseless_images)

# Iterate over sigma values
for i in range(len(sigmas)):
    # Collect PSNR values for each denoising method
    psnr_gaus_values = []
    psnr_tvm_values = []
    psnr_nlm_values = []

    for img_idx in range(num_images):
        original_image = noiseless_images[img_idx]
        
        # Indices are shifted by the number of sigma values
        corr_idx = img_idx * len(sigmas) + i
        
        # Calculate PSNR for each method and collect the results
        psnr_gaus_values.append(skmetrics.peak_signal_noise_ratio(original_image, gaus_filtered_img[corr_idx]))
        psnr_tvm_values.append(skmetrics.peak_signal_noise_ratio(original_image, tvm_filt_img[corr_idx]))
        psnr_nlm_values.append(skmetrics.peak_signal_noise_ratio(original_image, nlm_filt_img[corr_idx]))

    # Calculate the average PSNR for each denoising method at this sigma level
    avg_psnr_gaus.append(np.mean(psnr_gaus_values))
    avg_psnr_tvm.append(np.mean(psnr_tvm_values))
    avg_psnr_nlm.append(np.mean(psnr_nlm_values))

# Print the average PSNR results
print("Average PSNR for Gaussian:", avg_psnr_gaus)
print("Average PSNR for TVM:", avg_psnr_tvm)
print("Average PSNR for NLM:", avg_psnr_nlm)

Average PSNR for Gaussian: [27.580770423085287, 27.510415588728748, 27.08120434964656, 25.94627002985815, 23.33848647821407]
Average PSNR for TVM: [27.807596405944143, 27.783250393400532, 27.624376558073617, 26.759443063576423, 23.269607202589963]
Average PSNR for NLM: [26.45906248008095, 26.472532559370944, 26.545017210839642, 26.600820176012757, 17.91437130577588]


# Step 3: Plot the PSNR and SSIM for each noise level and denoising method.

## Instructions

- Generate a 2 x 6 group of subplots, each of which contains a bar graph.  The first row of three graphs should show the mean PSNR for each of the five noise levels for Gaussian filtering, TVM filtering and NLM filtering.  Sample output is given in the assignment PDF.  Your plots, subplot titles, axis labels, and tick labels should match the sample output.
- Make sure that the range of the y-axis is the same for all PSNR plots and the same for all SSIM plots.


In [1]:
# Write the code for this step here


# Step 4: Reflection

We'd expect the total variation minimization (TVM) to be slightly better than Gaussian filtering at most noise levels, but we do not see this.  We'd also expect the non-local means (NLM) filtering to outperform Gaussian filtering at all noise levels, and again we do not see this.  This is because we did all filtering using default parameters for the TVM and NLM filtering algorithms.  Why do the default parameters not produce expected results?

### Type your answer here in this block:



# Step 5: Improving the Results

## Instructions:

- Investigate the documentation for the denoise_tv_bregman() and denoise_nl_means() functions.   
- Determine more optimal parameters for these algorithms that will improve results across all noise levels.  You may make the values of parameters dependent on the known noise level of a given image (i.e. a parameter may be a function of the noise level).
- Hint: you can likely copy your code from Steps 2 and 3 and then add parameters to the denoising algoirthms.  
- The output of this step should have the same form as Step 3, but the bar heights will be different.

By adjusting the algorithm parameters judiciously, it should be possible to get TVM to at least very slightly outperform Gaussian filtering at all noise levels, and it should be possible to get NLM to greatly outperform both Gaussian filtering and TVM at low noise levels, and perform at least as well at high noise levels.

This step will be assessed based on how closely you match or exceed the expected results described above (see rubric on Canvas).

In [ ]:
# Write the code for this step here
